In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os

# Bu projede amaç fazla sayıda ki json dosyalarından anlamlı bir şekilde üzerinde işlem yapabileceğimiz csv formatında kaydedeilebilecek dataframe objelerini oluşturmaktır.
# Veri temizleme ve verileri sınıflandırma olarak iki ayrı kategori için de bu notebook dosyasının faydalı olabileceği düşünülmüştür.

"\n\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n\n\n"

### Main source (Kaynakça) : https://www.kaggle.com/code/xhlulu/cord-19-eda-parse-json-and-generate-clean-csv

# Loading the data

#### Veriyi yükleme aşamasında document_parses klasörü içerisinde pdf_json kalsöründe bulunan tüm json dosyalarından okuma yaparak bu dosyalardan gereken verilieri çıkartarak bir csv dosyas formatı haline getirmeyi amaçlıyoruz. 

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import json

import matplotlib.pyplot as plt
plt.style.use('ggplot')

Documents: Exploration
Let's first take a quick glance at the documents subset of the data. We will also use this opportunity to load all of the json files into a list of nested dictionaries (each dict is an article).

In [3]:
documents_dir = '/kaggle/input/CORD-19-research-challenge/document_parses/pdf_json/'
filenames = os.listdir(documents_dir)
print("Number of articles retrieved from biorxiv:", len(filenames))

Number of articles retrieved from biorxiv: 401214


In [4]:
all_files = []

# Because this notebook is running on Kaggle I will reduce the len of data
for filename in filenames[:2000]:
    filename = documents_dir + filename
    file = json.load(open(filename, 'rb'))
    all_files.append(file)

In [5]:
file = all_files[2]
print("Dictionary keys:", file.keys())

Dictionary keys: dict_keys(['paper_id', 'metadata', 'abstract', 'body_text', 'bib_entries', 'ref_entries', 'back_matter'])


Now we know the features that we will look through in documents. 

Bu aşamada dosyamızın hangi özelliklere sahip olduğunu gördük ve bu özelliklerden bazıları için detaylı bir şekilde inceleme yapmamız gerekiyor.

## Documents: Abstract

Abstract is very simple to understand.

In [6]:
from pprint import pprint
pprint(file['abstract'])

[{'cite_spans': [],
  'ref_spans': [],
  'section': 'Abstract',
  'text': 'Background Brazil ranks second worldwide in total number of '
          'COVID-19 cases and deaths. Understanding the possible socioeconomic '
          'and ethnic health inequities is particularly important given the '
          'diverse population and fragile political and economic situation. We '
          'aimed to characterise the COVID-19 pandemic in Brazil and assess '
          'variations in mortality according to region, ethnicity, '
          'comorbidities, and symptoms.'},
 {'cite_spans': [],
  'ref_spans': [],
  'section': 'Abstract',
  'text': 'Methods We conducted a cross-sectional observational study of '
          'COVID-19 hospital mortality using data from the SIVEP-Gripe '
          '(Sistema de Informação de Vigilância Epidemiológica da Gripe) '
          'dataset to characterise the COVID-19 pandemic in Brazil. In the '
          'study, we included hospitalised patients who had a positiv

#### **Note**: pprint function helps you to print data structures in a readable way. Useful for debugging code dealing with API requests, large JSON files

## Documents: Body Text

In [7]:
print("Type of the body text:",type(file['body_text']))
print("Length of the body text",len(file['body_text']))

Type of the body text: <class 'list'>
Length of the body text 32


In [8]:
print("Keys in the body text",file['body_text'][0].keys())

Keys in the body text dict_keys(['text', 'cite_spans', 'ref_spans', 'section'])


*We understood that each body text has section that we can group of*

In [9]:
print("body_text content:")
pprint(file['body_text'][:2], depth=2)

body_text content:
[{'cite_spans': [...],
  'ref_spans': [],
  'section': 'Introduction',
  'text': 'The COVID-19 pandemic has created an unprecedented worldwide strain '
          'on health care. Although early reports from east Asia and Europe '
          'meant that Brazil was well positioned to implement '
          'non-pharmaceutical interventions, Brazilians, like those in many '
          'low-income and middle-income countries, have limited access to '
          'testing and social security. 1, 2 Difficulties in testing make it '
          'harder to assess the growth of the pandemic, while limited access '
          'to social security can lead a sizable fraction of society to not '
          'engage in physical distancing. This situation has been further '
          'complicated by an unstable federal government 3 that has failed to '
          'support measures such as physical distancing and attempted to '
          'downplay the gravity of the pandemic, as has been well 

In [10]:
texts = [(d['section'],d['text']) for d in file['body_text']]
texts[0]

('Introduction',
 "The COVID-19 pandemic has created an unprecedented worldwide strain on health care. Although early reports from east Asia and Europe meant that Brazil was well positioned to implement non-pharmaceutical interventions, Brazilians, like those in many low-income and middle-income countries, have limited access to testing and social security. 1, 2 Difficulties in testing make it harder to assess the growth of the pandemic, while limited access to social security can lead a sizable fraction of society to not engage in physical distancing. This situation has been further complicated by an unstable federal government 3 that has failed to support measures such as physical distancing and attempted to downplay the gravity of the pandemic, as has been well publicised in the media. 4 Worryingly, as of June 23, 2020, Brazil ranks second worldwide in total number of COVID-19 cases and deaths, with a high estimated rate of transmission (effective reproduction number [R t ] 1·44). 5

In [11]:
text_di = {di['section']: "" for di in file['body_text']}
for section, text in texts:
    text_di[section] += text

pprint(list(text_di.keys()))

['Introduction',
 'Study design and population',
 'Research in context',
 'Added value of this study',
 'Implications of all the available evidence',
 'Statistical analysis',
 'Role of the funding source',
 'Results',
 'Discussion']


format_body fonksiyonu ileride tüm dosyaların sahip olduğu yazıları formatlı bir hale getirmede bize yardımcı olucaktır.

In [154]:
def format_body(body_text):
    texts = [(di['section'], di['text']) for di in body_text]
    texts_di = {di['section']: "" for di in body_text}
    
    for section, text in texts:
        texts_di[section] += text

    body = ""

    for section, text in texts_di.items():
        body += section
        body += "\n\n"
        body += text
        body += "\n\n"
    
    return body

In [14]:
body = ""

for section, text in text_di.items():
    body+="\033[1m" 
    body+=section
    body+="\033[0m"
    body+="\n\t"
    body+=text
    body+="\n\n"

print(body)
    

Introduction
	The COVID-19 pandemic has created an unprecedented worldwide strain on health care. Although early reports from east Asia and Europe meant that Brazil was well positioned to implement non-pharmaceutical interventions, Brazilians, like those in many low-income and middle-income countries, have limited access to testing and social security. 1, 2 Difficulties in testing make it harder to assess the growth of the pandemic, while limited access to social security can lead a sizable fraction of society to not engage in physical distancing. This situation has been further complicated by an unstable federal government 3 that has failed to support measures such as physical distancing and attempted to downplay the gravity of the pandemic, as has been well publicised in the media. 4 Worryingly, as of June 23, 2020, Brazil ranks second worldwide in total number of COVID-19 cases and deaths, with a high estimated rate of transmission (effective reproduction number [R t ] 1·44). 5 Worl

In [157]:
print(format_body(file['body_text']))

Introduction

The COVID-19 pandemic has created an unprecedented worldwide strain on health care. Although early reports from east Asia and Europe meant that Brazil was well positioned to implement non-pharmaceutical interventions, Brazilians, like those in many low-income and middle-income countries, have limited access to testing and social security. 1, 2 Difficulties in testing make it harder to assess the growth of the pandemic, while limited access to social security can lead a sizable fraction of society to not engage in physical distancing. This situation has been further complicated by an unstable federal government 3 that has failed to support measures such as physical distancing and attempted to downplay the gravity of the pandemic, as has been well publicised in the media. 4 Worryingly, as of June 23, 2020, Brazil ranks second worldwide in total number of COVID-19 cases and deaths, with a high estimated rate of transmission (effective reproduction number [R t ] 1·44). 5 Worl

## Documents: Metadata

In [16]:
print(file['metadata'].keys())

dict_keys(['title', 'authors'])


In [17]:
print(file['metadata']['title'])

Ethnic and regional variations in hospital mortality from COVID-19 in Brazil: a cross-sectional observational study


In [19]:
pprint(file['metadata']['authors'])

[{'affiliation': {},
  'email': '',
  'first': 'Núcleo',
  'last': 'De Astrofísica E Cosmologia',
  'middle': [],
  'suffix': ''},
 {'affiliation': {},
  'email': '',
  'first': 'P',
  'last': 'Baqui',
  'middle': [],
  'suffix': ''},
 {'affiliation': {},
  'email': 'marra@cosmo-ufes.org',
  'first': 'V',
  'last': 'Marra',
  'middle': [],
  'suffix': ''},
 {'affiliation': {},
  'email': '',
  'first': 'Phd',
  'last': '',
  'middle': [],
  'suffix': ''},
 {'affiliation': {},
  'email': '',
  'first': 'M',
  'last': 'Van Der Schaar',
  'middle': [],
  'suffix': ''},
 {'affiliation': {},
  'email': '',
  'first': 'Valerio',
  'last': 'Marra',
  'middle': [],
  'suffix': ''},
 {'affiliation': {},
  'email': '',
  'first': 'Pedro',
  'last': 'Baqui',
  'middle': [],
  'suffix': ''},
 {'affiliation': {},
  'email': '',
  'first': 'Ioana',
  'last': 'Bica',
  'middle': [],
  'suffix': ''},
 {'affiliation': {},
  'email': '',
  'first': 'Ari',
  'last': 'Ercole',
  'middle': [],
  'suffix': 

In [128]:
def format_name(author):
    middle_name = " ".join(author['middle'])
    # if we do not control middle then it will return an error when there is no middle name
    if author['middle']:
        return " ".join([author['first'], middle_name, author['last']])
    else:
        return " ".join([author['first'], author['last']])


def format_affiliation(affiliation):
    text = []
    location = affiliation.get('location')
    if location:
        text.extend(list(affiliation['location'].values()))
    
    institution = affiliation.get('institution')
    if institution:
        text = [institution] + text
    return ", ".join(text)

def format_authors(authors, with_affiliation=False):
    name_ls = []
    
    for author in authors:
        name = format_name(author)
        if with_affiliation:
            affiliation = format_affiliation(author['affiliation'])
            if affiliation:
                name_ls.append(f"{name} ({affiliation})")
            else:
                name_ls.append(f"{name}")
        else:
            name_ls.append(name)
            
    return ", ".join(name_ls)

In [129]:
authors = file['metadata']['authors']
format_authors(authors,with_affiliation=False)

'Núcleo De Astrofísica E Cosmologia, P Baqui, V Marra, Phd , M Van Der Schaar, Valerio Marra, Pedro Baqui, Ioana Bica, Ari Ercole, Mihaela Van Der Schaar'

In [134]:
file1 = all_files[50]
authors = file1['metadata']['authors']
format_authors(authors,with_affiliation=True)

'Dat Quoc Nguyen (VinAI Research, Vietnam), Thanh Vu (CSIRO, Australia), Anh Tuan Nguyen (VinAI Research, Vietnam)'

## Documents: bibliography (kaynakça)

In [135]:
bibs = list(file['bib_entries'].values())
pprint(bibs, depth=4)

[{'authors': [{'first': 'F', 'last': 'Tavares', 'middle': [], 'suffix': ''},
              {'first': 'G', 'last': 'Betti', 'middle': [], 'suffix': ''}],
  'issn': '',
  'other_ids': {},
  'pages': '',
  'ref_id': 'b1',
  'title': 'Vulnerability, poverty and COVID-19: risk factors and deprivations '
           'in Brazil',
  'venue': '',
  'volume': '',
  'year': 2020},
 {'authors': [],
  'issn': '',
  'other_ids': {},
  'pages': '',
  'ref_id': 'b2',
  'title': 'COVID-19 in Brazil',
  'venue': 'The Lancet',
  'volume': '395',
  'year': 2020},
 {'authors': [{'first': 'T', 'last': 'Phillips', 'middle': [], 'suffix': ''},
              {'first': '', 'last': 'Brazil', 'middle': [], 'suffix': ''}],
  'issn': '',
  'other_ids': {},
  'pages': '',
  'ref_id': 'b3',
  'title': "Bolsonaro's defiance of distancing criticized by health minister. "
           '2020. The Guardian',
  'venue': '',
  'volume': '',
  'year': 2020},
 {'authors': [],
  'issn': '',
  'other_ids': {},
  'pages': '',
  're

#### [INFO]: Deep copy makes that any changes made to a copy of the object do not reflect in the original object. 
For more info: https://www.geeksforgeeks.org/copy-python-deep-copy-shallow-copy/#:~:text=What%20is%20Deep%20copy%20in,objects%20found%20in%20the%20original.

In [150]:
from copy import deepcopy
def format_bib(bibs):
    if type(bibs) == dict:
        bibs = list(bibs.values())
    bibs = deepcopy(bibs)
    formatted = []
    
    for bib in bibs:
        bib['authors'] = format_authors(
            bib['authors'], 
            with_affiliation=False
        )
        formatted_ls = [str(bib[k]) for k in ['title', 'authors','pages' ,'venue', 'year']]
        formatted.append(", ".join(formatted_ls))

    return "; ".join(formatted)

In [159]:
pprint(format_bib(bibs))

('Vulnerability, poverty and COVID-19: risk factors and deprivations in '
 'Brazil, F Tavares, G Betti, , , 2020; COVID-19 in Brazil, , , The Lancet, '
 "2020; Bolsonaro's defiance of distancing criticized by health minister. "
 '2020. The Guardian, T Phillips,  Brazil, , , 2020; Imperial College COVID-19 '
 'response team. Short-term forecasts of COVID-19 deaths in multiple '
 'countries, , , , 2020; Evidence mounts on the disproportionate effect of '
 'COVID-19 on ethnic minorities, T Kirby, 547--595, Lancet Respir Med, 2020; '
 'Ethnicity and COVID-19: an urgent public health research priority, M Pareek, '
 'M N Bangash, N Pareek, 1421--1443, Lancet, 2020; Open-SAFELY: factors '
 'associated with COVID-19-related hospital death in the linked electronic '
 'health records of 17 million adult NHS patients, E Williamson, A J Walker, K '
 'J Bhaskaran, , medRxiv, 2020; US Centers for Disease Control and Prevention. '
 'COVID-19 in racial and ethnic minority groups. 2020, , , , 2020; The

## Generate CSV From Documents

In [164]:
from tqdm.notebook import tqdm
cleaned_files = []

for file in tqdm(all_files):
    features = [
        file['paper_id'],
        file['metadata']['title'],
        format_authors(file['metadata']['authors']),
        format_authors(file['metadata']['authors'], 
                       with_affiliation=True),
        format_body(file['abstract']),
        format_body(file['body_text']),
        format_bib(file['bib_entries']),
        file['metadata']['authors'],
        file['bib_entries']
    ]
    
    cleaned_files.append(features)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [165]:
# To turn pandas dataframe format we are going to write column names
col_names = ['paper_id','title','authors','afilliations','abstract','text','bibliography','raw_authors','raw_bibliography']
# Türkçe sütunları anlamlandırabilmek için
col_names_tr = ['makale_id','baslik','yazarlar','kurumlar','özet','yazi','kaynakca','yazar_formatsiz','kaynakca_formatsiz']

In [166]:
clean_df = pd.DataFrame(cleaned_files,columns=col_names)
clean_df.head()

,paper_id,title,authors,afilliations,abstract,text,bibliography,raw_authors,raw_bibliography
0,8f97e16f3842e4bbd2d5d1c0c95ac1e31993ec68,An update on excess mortality in the second ye...,"Asta Wirtsch, Sozialstat Arch, Giacomo De Nico...","Asta Wirtsch, Sozialstat Arch, Giacomo De Nico...","Abstract\n\nIn this short note, we apply the m...","\n\nExpected deaths by year, represented by bl...",On assessing excess mortality in Germany durin...,"[{'first': 'Asta', 'middle': [], 'last': 'Wirt...","{'BIBREF0': {'ref_id': 'b0', 'title': 'On asse..."
1,8187ea360c53a56ca2c579d758a5d6aa67716836,,,,,COVID-19-negative psychiatric units: Mitigatin...,"Protecting People, , , CDC COVID data tracker:...",[],"{'BIBREF0': {'ref_id': 'b0', 'title': 'Protect..."
2,a0d063dca746b135afe0451ce0b3bb1e06cf15ae,Ethnic and regional variations in hospital mor...,"Núcleo De Astrofísica E Cosmologia, P Baqui, V...","Núcleo De Astrofísica E Cosmologia, P Baqui, V...",Abstract\n\nBackground Brazil ranks second wor...,Introduction\n\nThe COVID-19 pandemic has crea...,"Vulnerability, poverty and COVID-19: risk fact...","[{'first': 'Núcleo', 'middle': [], 'last': 'De...","{'BIBREF1': {'ref_id': 'b1', 'title': 'Vulnera..."
3,edb294108440787c9f074483fd3c953a83e53622,Corona! Die Krise der Verschlankung und ihre F...,"Z Arb, Wiss, Irene Raehlmann","Z Arb, Wiss, Irene Raehlmann",,Einleitung\n\nDie Corona-Pandemie ist eine Gef...,"Unternehmen bereiten Notfallpläne vor, C Busse...","[{'first': 'Z', 'middle': [], 'last': 'Arb', '...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Unterne..."
4,e0777fb5df224525ee1b06008582b084c1b6b13b,"Association between central blood pressure, ar...","Feziwe Mpondo, Ashleigh Craig, Andrea Kolkenbe...",Feziwe Mpondo (University of the Witwatersrand...,Abstract\n\nThe burden of hypertension in Sout...,Introduction\n\nRaised blood pressure (BP) and...,"Global, regional, and national burden of cardi...","[{'first': 'Feziwe', 'middle': [], 'last': 'Mp...","{'BIBREF0': {'ref_id': 'b0', 'title': 'Global,..."
